<a href="https://colab.research.google.com/github/vfrantc/weather_experiments/blob/main/detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Thu Jul 28 20:33:31 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    24W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [3]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [4]:
# install dependencies: (use cu111 because colab has CUDA 11.1)
!pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html

# install mmcv-full thus we could use CUDA operators
!pip install mmcv-full==1.6.0 -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.9.0/index.html

# Install mmdetection
!rm -rf mmdetection
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection

!pip install -e .
!pip install -e .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |█████████████                   | 834.1 MB 1.2 MB/s eta 0:16:41tcmalloc: large alloc 1147494400 bytes == 0x38d2c000 @  0x7f985408e615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |████████████████▌               | 1055.7 MB 1.2 MB/s eta 0:14:00tcmalloc: large alloc 1434370048 bytes == 0x7d382000 @  0x7f985408e615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |█████

# RUN DETECTOR ON UNPROCESSED IMAGES

In [5]:
!mkdir checkpoints
!wget -c https://download.openmmlab.com/mmdetection/v2.0/scnet/scnet_x101_64x4d_fpn_20e_coco/scnet_x101_64x4d_fpn_20e_coco-fb09dec9.pth \
      -O checkpoints/scnet_x101_64x4d_fpn_20e_coco-fb09dec9.pth

--2022-07-28 20:37:53--  https://download.openmmlab.com/mmdetection/v2.0/scnet/scnet_x101_64x4d_fpn_20e_coco/scnet_x101_64x4d_fpn_20e_coco-fb09dec9.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 47.254.186.219
Connecting to download.openmmlab.com (download.openmmlab.com)|47.254.186.219|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 611860651 (584M) [application/octet-stream]
Saving to: ‘checkpoints/scnet_x101_64x4d_fpn_20e_coco-fb09dec9.pth’

checkpoints/scnet_x 100%[===================>] 583.52M  7.63MB/s    in 79s     

2022-07-28 20:39:14 (7.42 MB/s) - ‘checkpoints/scnet_x101_64x4d_fpn_20e_coco-fb09dec9.pth’ saved [611860651/611860651]



In [6]:
import os
from glob import glob
from tqdm.notebook import tqdm

import numpy as np
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.datasets import build_dataloader
from mmdet.apis import single_gpu_test
from mmcv.parallel import MMDataParallel
from mmdet.apis import inference_detector, init_detector, show_result_pyplot

config = 'configs/scnet/scnet_x101_64x4d_fpn_20e_coco.py'
checkpoint = 'checkpoints/scnet_x101_64x4d_fpn_20e_coco-fb09dec9.pth'
model = init_detector(config, checkpoint, device='cuda:0')
model = MMDataParallel(model, device_ids=[0])

/content/mmdetection/mmdet/models/losses/cross_entropy_loss.py:240: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  'Default ``avg_non_ignore`` is False, if you would like to '


load checkpoint from local path: checkpoints/scnet_x101_64x4d_fpn_20e_coco-fb09dec9.pth


In [7]:
def fix_outputs(fix_outputs):
    new_outputs = []
    for output in outputs:
        new_output = []
        for i in range(80):
            new_output.append(np.array([], dtype=np.float32).reshape(0, 5))
        for i in [0, 1, 2, 3, 5]:
            new_output[i] = output[0][i]
        new_output[2] = np.vstack([new_output[2], output[0][i]])
        new_outputs.append(new_output)
    return new_outputs

In [8]:
def create_dataset(folder):
  cfg = Config.fromfile('./configs/scnet/scnet_x101_64x4d_fpn_20e_coco.py')
  cfg.dataset_type = 'CocoDataset'
  PREFIX = folder
  cfg.data.test.img_prefix = folder
  cfg.data.test.ann_file = os.path.join(PREFIX, 'new_test.json')
  cfg.data.test.type = 'CocoDataset'
  dataset = build_dataset(cfg.data.test)
  #dataset.CLASSES = classes
  #dataset.cat_ids = list(range(len(dataset.CLASSES)))
  data_loader = build_dataloader(dataset, samples_per_gpu=1, workers_per_gpu=2, dist=False, shuffle=False)
  return dataset, data_loader

# Original

In [11]:
!cp /content/drive/MyDrive/detect_dawn.zip .
!unzip -q detect_dawn.zip

replace dataset/sand/new_test.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [14]:
!ls dataset/fog

Annotations  ImageSets	JPEGImages  new_test.json


In [ ]:
dataset, data_loader = create_dataset(f'dataset/fog')
dataset.cat_ids.append(91)
outputs = single_gpu_test(model, data_loader, False, '', 0.3)
metric = dataset.evaluate(fix_outputs(outputs), metric='bbox')

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
[                                                  ] 0/299, elapsed: 0s, ETA:

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


[>>>>>>>>>>>>>>>>>>>>>>>>>    ] 268/299, 2.4 task/s, elapsed: 111s, ETA:    13s

# NL

In [ ]:
method = 'nl'
!mkdir out_{method}
dataset, data_loader = create_dataset(f'./{method}')
dataset.cat_ids.append(91)
outputs = single_gpu_test(model, data_loader, False, '', 0.3)
metric = dataset.evaluate(fix_outputs(outputs), metric='bbox')

mkdir: cannot create directory ‘out_nl’: File exists
loading annotations into memory...
Done (t=0.48s)
creating index...
index created!
[>>>>>>>>>>>>>>>>>>>>>>>>>>] 4320/4320, 1.9 task/s, elapsed: 2317s, ETA:     0s
Evaluating bbox...
Loading and preparing results...
DONE (t=1.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=73.12s).
Accumulating evaluation results...
DONE (t=8.93s).

 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.352
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.555
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.381
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.133
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.330
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.478
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=

In [ ]:
method = 'wavelet_unet'
!mkdir out_{method}
dataset, data_loader = create_dataset(f'./{method}')
dataset.cat_ids.append(91)
outputs = single_gpu_test(model, data_loader, False, '', 0.3)
metric = dataset.evaluate(fix_outputs(outputs), metric='bbox')

loading annotations into memory...
Done (t=0.37s)
creating index...
index created!
[>>>>>>>>>>>>>>>>>>>>>>>>>>] 4320/4320, 2.0 task/s, elapsed: 2144s, ETA:     0s
Evaluating bbox...
Loading and preparing results...
DONE (t=0.85s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=54.56s).
Accumulating evaluation results...
DONE (t=6.75s).

 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.350
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.558
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.371
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.146
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.332
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.467
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.434
 Average Recall     (AR) @[ IoU=0.50:0.

### gca

In [ ]:
!cp /content/drive/MyDrive/new_test.json out_gca

In [ ]:
method = 'out_gca'
!mkdir out_{method}
dataset, data_loader = create_dataset(f'./{method}')
outputs = single_gpu_test(model, data_loader, False, '', 0.3)

mkdir: cannot create directory ‘out_out_gca’: File exists
loading annotations into memory...
Done (t=0.15s)
creating index...
index created!
[>>>>>>>>>>>>>>>>>>>>>>>>>>] 4320/4320, 1.8 task/s, elapsed: 2363s, ETA:     0s

In [ ]:
# how to post-process, and how to change the json

In [ ]:
# post-process the detection to make sure that we use only the correct ones

In [ ]:
dataset.cat_ids.append(91)
metric = dataset.evaluate(fix_outputs(outputs), metric='bbox')


Evaluating bbox...
Loading and preparing results...
DONE (t=1.20s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=53.88s).
Accumulating evaluation results...
DONE (t=6.96s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.363
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.567
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.390
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.155
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.346
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.476
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.449
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.449
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.449
 Average Recall     (AR) @[ IoU=0.50:0.95

In [ ]:
!tar czf detected_gca.tar.gz out_out_gca/
!cp detected_gca.tar.gz /content/drive/MyDrive/new_detect

### dcp


In [ ]:
method = 'dcp_coco'
dataset, data_loader = create_dataset(f'./{method}')
dataset.cat_ids.append(91)
outputs = single_gpu_test(model, data_loader, False, '', 0.3)
metric = dataset.evaluate(fix_outputs(outputs), metric='bbox')

loading annotations into memory...
Done (t=0.87s)
creating index...
index created!
[>>>>>>>>>>>>>>>>>>>>>>>>>>] 4320/4320, 1.9 task/s, elapsed: 2278s, ETA:     0s
Evaluating bbox...
Loading and preparing results...
DONE (t=1.19s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=53.21s).
Accumulating evaluation results...
DONE (t=6.77s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.355
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.560
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.382
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.144
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.335
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.475
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.438
 Average Recall     (AR) @[ IoU=0.50:0.9

In [ ]:
# !tar czf detected_dcp.tar.gz out_dcp_coco/
# !cp -r detected_dcp.tar.gz /content/drive/MyDrive/new_detect

### Refined

In [ ]:
!cp /content/drive/MyDrive/new_test.json out_refined

In [ ]:
!ls out_refined/JPEGImages

AM_Bing_211_dehz.png	    HazyDr_Google_188_dehz.png	MLS_Bing_034_dehz.png
AM_Bing_217_dehz.png	    HazyDr_Google_240_dehz.png	MLS_Bing_036_dehz.png
AM_Bing_222_dehz.png	    HazyDr_Google_277_dehz.png	MLS_Bing_046_dehz.png
AM_Bing_229_dehz.png	    HazyDr_Google_289_dehz.png	MLS_Bing_068_dehz.png
AM_Bing_232_dehz.png	    HazyDr_Google_298_dehz.png	MLS_Bing_085_dehz.png
AM_Bing_242_dehz.png	    HazyDr_Google_324_dehz.png	MLS_Bing_095_dehz.png
AM_Bing_243_dehz.png	    HazyDr_Google_359_dehz.png	MLS_Bing_108_dehz.png
AM_Bing_274_dehz.png	    HazyDr_Google_384_dehz.png	MLS_Bing_115_dehz.png
AM_Bing_318_dehz.png	    HazyDr_Google_475_dehz.png	MLS_Bing_118_dehz.png
AM_Bing_422_dehz.png	    HazyDr_Google_476_dehz.png	MLS_Bing_123_dehz.png
AM_Bing_464_dehz.png	    HazyDr_Google_602_dehz.png	MLS_Bing_130_dehz.png
AM_Bing_536_dehz.png	    HazyDr_Google_622_dehz.png	MLS_Bing_132_dehz.png
AM_Bing_640_dehz.png	    HazyDr_Google_647_dehz.png	MLS_Bing_141_dehz.png
AM_Bing_727_dehz.png	    HazyDr_Google

In [ ]:
!ls out_refined/JPEGImages 

AM_Bing_211_dehz.png	    HazyDr_Google_188_dehz.png	MLS_Bing_034_dehz.png
AM_Bing_217_dehz.png	    HazyDr_Google_240_dehz.png	MLS_Bing_036_dehz.png
AM_Bing_222_dehz.png	    HazyDr_Google_277_dehz.png	MLS_Bing_046_dehz.png
AM_Bing_229_dehz.png	    HazyDr_Google_289_dehz.png	MLS_Bing_068_dehz.png
AM_Bing_232_dehz.png	    HazyDr_Google_298_dehz.png	MLS_Bing_085_dehz.png
AM_Bing_242_dehz.png	    HazyDr_Google_324_dehz.png	MLS_Bing_095_dehz.png
AM_Bing_243_dehz.png	    HazyDr_Google_359_dehz.png	MLS_Bing_108_dehz.png
AM_Bing_274_dehz.png	    HazyDr_Google_384_dehz.png	MLS_Bing_115_dehz.png
AM_Bing_318_dehz.png	    HazyDr_Google_475_dehz.png	MLS_Bing_118_dehz.png
AM_Bing_422_dehz.png	    HazyDr_Google_476_dehz.png	MLS_Bing_123_dehz.png
AM_Bing_464_dehz.png	    HazyDr_Google_602_dehz.png	MLS_Bing_130_dehz.png
AM_Bing_536_dehz.png	    HazyDr_Google_622_dehz.png	MLS_Bing_132_dehz.png
AM_Bing_640_dehz.png	    HazyDr_Google_647_dehz.png	MLS_Bing_141_dehz.png
AM_Bing_727_dehz.png	    HazyDr_Google

In [ ]:
!ls out_refined/JPEGImages | wc -l

4322


In [ ]:
!ls msnet/JPEGImages | wc -l

4322


In [ ]:
!ls out_refined/JPEGImages

AM_Bing_211_dehz.png	    HazyDr_Google_188_dehz.png	MLS_Bing_034_dehz.png
AM_Bing_217_dehz.png	    HazyDr_Google_240_dehz.png	MLS_Bing_036_dehz.png
AM_Bing_222_dehz.png	    HazyDr_Google_277_dehz.png	MLS_Bing_046_dehz.png
AM_Bing_229_dehz.png	    HazyDr_Google_289_dehz.png	MLS_Bing_068_dehz.png
AM_Bing_232_dehz.png	    HazyDr_Google_298_dehz.png	MLS_Bing_085_dehz.png
AM_Bing_242_dehz.png	    HazyDr_Google_324_dehz.png	MLS_Bing_095_dehz.png
AM_Bing_243_dehz.png	    HazyDr_Google_359_dehz.png	MLS_Bing_108_dehz.png
AM_Bing_274_dehz.png	    HazyDr_Google_384_dehz.png	MLS_Bing_115_dehz.png
AM_Bing_318_dehz.png	    HazyDr_Google_475_dehz.png	MLS_Bing_118_dehz.png
AM_Bing_422_dehz.png	    HazyDr_Google_476_dehz.png	MLS_Bing_123_dehz.png
AM_Bing_464_dehz.png	    HazyDr_Google_602_dehz.png	MLS_Bing_130_dehz.png
AM_Bing_536_dehz.png	    HazyDr_Google_622_dehz.png	MLS_Bing_132_dehz.png
AM_Bing_640_dehz.png	    HazyDr_Google_647_dehz.png	MLS_Bing_141_dehz.png
AM_Bing_727_dehz.png	    HazyDr_Google

In [ ]:
from glob import glob
from shutil import move

In [ ]:
move?

In [ ]:
for imname in glob('out_refined/JPEGImages/*.png'):
  new_imname = imname.replace('_dehz.png', '.png')
  move(imname, new_imname)

In [ ]:
method = 'out_refined'
!rm -rf out_out_refined
!mkdir out_{method}
dataset, data_loader = create_dataset(f'./{method}')
dataset.cat_ids.append(91)
outputs = single_gpu_test(model, data_loader, False, '', 0.3)
metric = dataset.evaluate(outputs, metric='bbox')

loading annotations into memory...
Done (t=0.65s)
creating index...
index created!
[>>>>>>>>>>>>>>>>>>>>>>>>>>] 4320/4320, 2.0 task/s, elapsed: 2147s, ETA:     0s
Evaluating bbox...
Loading and preparing results...
DONE (t=1.28s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=63.32s).
Accumulating evaluation results...
DONE (t=8.82s).

 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.342
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.550
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.362
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.119
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.321
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.479
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.428
 Average Recall     (AR) @[ IoU=0.50:0.

In [ ]:
# !tar czf detected_refined.tar.gz out_out_refined/
# !cp -r detected_refined.tar.gz /content/drive/MyDrive/new_detect

# Dehazenet

In [ ]:
method = 'dehazenet_coco'
!mkdir out_{method}
dataset, data_loader = create_dataset(f'./{method}')
dataset.cat_ids.append(91)
outputs = single_gpu_test(model, data_loader, False, '', 0.3)
metric = dataset.evaluate(fix_outputs(outputs), metric='bbox')

loading annotations into memory...
Done (t=0.16s)
creating index...
index created!
[>>>>>>>>>>>>>>>>>>>>>>>>>>] 4320/4320, 1.9 task/s, elapsed: 2231s, ETA:     0s
Evaluating bbox...
Loading and preparing results...
DONE (t=0.28s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=54.64s).
Accumulating evaluation results...
DONE (t=6.61s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.351
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.561
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.375
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.143
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.333
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.470
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.436
 Average Recall     (AR) @[ IoU=0.50:0.9

In [ ]:
# !tar czf detected_dehazenet.tar out_dehazenet/
# !cp -r detected_dehazenet.zip /content/drive/MyDrive/new_detect

# IDE

In [ ]:
method = 'dide_coco'
!cp /content/drive/MyDrive/new_test.json dide_coco
!mkdir out_{method}
dataset, data_loader = create_dataset(f'./{method}')
dataset.cat_ids.append(91)
outputs = single_gpu_test(model, data_loader, False, '', 0.3)
metric = dataset.evaluate(fix_outputs(outputs), metric='bbox')

loading annotations into memory...
Done (t=0.15s)
creating index...
index created!
[>>>>>>>>>>>>>>>>>>>>>>>>>>] 4320/4320, 1.9 task/s, elapsed: 2221s, ETA:     0s
Evaluating bbox...
Loading and preparing results...
DONE (t=1.25s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=54.63s).
Accumulating evaluation results...
DONE (t=7.13s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.353
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.559
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.382
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.140
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.336
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.473
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.442
 Average Recall     (AR) @[ IoU=0.50:0.9

In [ ]:
# !tar czf detected_ide.tar.gz out_ide_coco/
# !cp -r detected_ide.tar.gz /content/drive/MyDrive/new_detect

# RTTS

In [ ]:
!rm -rf rtts_coco
!cp /content/drive/MyDrive/haze/data/RTTS.zip .
!unzip -q RTTS.zip
!mv RTTS rtts_coco 
!cp /content/drive/MyDrive/new_test.json rtts_coco

In [ ]:
method = 'rtts_coco'
!mkdir out_{method}
dataset, data_loader = create_dataset(f'./{method}')
dataset.cat_ids.append(91)
outputs = single_gpu_test(model, data_loader, False, '', 0.3)
metric = dataset.evaluate(fix_outputs(outputs), metric='bbox')

loading annotations into memory...
Done (t=0.16s)
creating index...
index created!
[>>>>>>>>>>>>>>>>>>>>>>>>>>] 4320/4320, 2.0 task/s, elapsed: 2174s, ETA:     0s
Evaluating bbox...
Loading and preparing results...
DONE (t=1.16s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=55.34s).
Accumulating evaluation results...
DONE (t=6.71s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.354
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.562
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.378
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.145
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.337
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.473
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.441
 Average Recall     (AR) @[ IoU=0.50:0.9

In [ ]:
# !tar czf detected_rtts_coco.tar.gz out_rtts_coco/
# !cp -r detected_rtts_coco.tar.gz /content/drive/MyDrive/new_detect

# AOD

In [ ]:
method = 'aod_coco'
dataset, data_loader = create_dataset(f'./{method}')
dataset.cat_ids.append(91)
outputs = single_gpu_test(model, data_loader, False, '', 0.3)
metric = dataset.evaluate(fix_outputs(outputs), metric='bbox')

loading annotations into memory...
Done (t=0.15s)
creating index...
index created!
[>>>>>>>>>>>>>>>>>>>>>>>>>>] 4320/4320, 1.9 task/s, elapsed: 2254s, ETA:     0s
Evaluating bbox...
Loading and preparing results...
DONE (t=1.22s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=56.04s).
Accumulating evaluation results...
DONE (t=6.96s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.334
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.540
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.355
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.131
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.317
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.453
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.419
 Average Recall     (AR) @[ IoU=0.50:0.9

# EPDN

In [ ]:
method = 'out_epdn'
dataset, data_loader = create_dataset(f'./{method}')
dataset.cat_ids.append(91)
outputs = single_gpu_test(model, data_loader, False, '', 0.3)
metric = dataset.evaluate(fix_outputs(outputs), metric='bbox')

loading annotations into memory...
Done (t=1.90s)
creating index...
index created!
[>>>>>>>>>>>>>>>>>>>>>>>>>>] 4320/4320, 2.0 task/s, elapsed: 2203s, ETA:     0s
Evaluating bbox...
Loading and preparing results...
DONE (t=0.91s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=55.97s).
Accumulating evaluation results...
DONE (t=7.06s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.202
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.442
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.160
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.012
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.155
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.389
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.296
 Average Recall     (AR) @[ IoU=0.50:0.9

In [ ]:
from glob import glob
from shutil import move

for fname in glob('out_epdn/JPEGImages/*.png'):
  move(fname, "_".join(fname.split("_")[:-1]) + '.png')
  # move(fname, "_")